In [1]:
import os

# 开启 LangSmith 跟踪，便于调试和查看详细执行信息
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ChatBot2"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# 定义状态类型，继承自 TypedDict，并使用 add_messages 函数将消息追加到现有列表
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 创建一个状态图对象，传入状态定义
graph_builder = StateGraph(State)

In [3]:
from langchain_openai import ChatOpenAI

# 初始化一个 GPT-4o-mini 模型
# chat_model = ChatOpenAI(model="gpt-4o-mini")
chat_model = ChatOpenAI(
    model="deepseek-chat",  # DeepSeek 指定的模型名称
    base_url="https://api.deepseek.com",  # DeepSeek 的 API 端点
    api_key="sk-3e5ff44e82b745a7ab7a748b806951c2",  # 替换为你的 DeepSeek API Key
    temperature=0.5,
    max_tokens=4000
)
# 定义聊天机器人的节点函数，接收当前状态并返回更新的消息列表
def chatbot(state: State):
    return {"messages": [chat_model.invoke(state["messages"])]}

# 第一个参数是唯一的节点名称，第二个参数是每次节点被调用时的函数或对象
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
# 编译状态图并生成可执行图对象
graph = graph_builder.compile()

In [4]:
# 开始一个简单的聊天循环
while True:
    # 获取用户输入
    user_input = input("User: ")
    
    # 可以随时通过输入 "quit"、"exit" 或 "q" 退出聊天循环
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")  # 打印告别信息
        break  # 结束循环，退出聊天

    # 将每次用户输入的内容传递给 graph.stream，用于聊天机器人状态处理
    # "messages": ("user", user_input) 表示传递的消息是用户输入的内容
    for event in graph.stream({"messages": ("user", user_input)}):
        
        # 遍历每个事件的值
        for value in event.values():
            # 打印输出 chatbot 生成的最新消息
            print("Assistant:", value["messages"][-1].content)

User:  火箭队今天赢球了吗


Assistant: 要查看火箭队今天是否赢球，您可以参考以下步骤获取最新信息：

1. **实时比分平台**  
   - **NBA官网**（[www.nba.com](https://www.nba.com)）或官方App会更新实时赛果。  
   - **ESPN**、**腾讯体育**等体育网站提供赛程和比分。  

2. **今日是否有比赛**  
   - 火箭队的赛程受赛季时间影响（NBA常规赛通常为10月至次年4月，季后赛持续到6月）。若当前为休赛期，则无比赛。

3. **快捷查询方法**  
   - 直接搜索“火箭队今天比赛结果” + 当前日期（如：2023年12月25日），谷歌或百度会显示即时赛果。  

**若您需要，我可以帮您查找最新比赛信息！** 请告知具体日期或确认当前是否在NBA赛季中。


User:  q


Goodbye!


### 2. 定义工具

In [5]:
import requests
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_openai import ChatOpenAI
from langchain.tools import tool

BOCHA_API_KEY = "sk-48eb0b2a54954b8c92a3fa0748d124b0"

# 定义Bocha Web Search工具
@tool
def bocha_websearch_tool(query: str, count: int = 10) -> str:
    """
    使用Bocha Web Search API 进行网页搜索。

    参数:
    - query: 搜索关键词
    - freshness: 搜索的时间范围
    - summary: 是否显示文本摘要
    - count: 返回的搜索结果数量

    返回:
    - 搜索结果的详细信息，包括网页标题、网页URL、网页摘要、网站名称、网站Icon、网页发布时间等。
    """
    
    url = 'https://api.bochaai.com/v1/web-search'
    headers = {
        'Authorization': f'Bearer {BOCHA_API_KEY}',  # 请替换为你的API密钥
        'Content-Type': 'application/json'
    }
    data = {
        "query": query,
        "freshness": "noLimit", # 搜索的时间范围，例如 "oneDay", "oneWeek", "oneMonth", "oneYear", "noLimit"
        "summary": True, # 是否返回长文本摘要
        "count": count
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        json_response = response.json()
        try:
            if json_response["code"] != 200 or not json_response["data"]:
                return f"搜索API请求失败，原因是: {response.msg or '未知错误'}"
            
            webpages = json_response["data"]["webPages"]["value"]
            if not webpages:
                return "未找到相关结果。"
            formatted_results = ""
            for idx, page in enumerate(webpages, start=1):
                formatted_results += (
                    f"引用: {idx}\n"
                    f"标题: {page['name']}\n"
                    f"URL: {page['url']}\n"
                    f"摘要: {page['summary']}\n"
                    f"网站名称: {page['siteName']}\n"
                    f"网站图标: {page['siteIcon']}\n"
                    f"发布时间: {page['dateLastCrawled']}\n\n"
                )
            return formatted_results.strip()
        except Exception as e:
            return f"搜索API请求失败，原因是：搜索结果解析失败 {str(e)}"
    else:
        return f"搜索API请求失败，状态码: {response.status_code}, 错误信息: {response.text}"


In [8]:
from typing import Annotated
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages

# 定义状态
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# 初始化 LLM 并绑定搜索工具
chat_model = ChatOpenAI(
    model="deepseek-chat",  # DeepSeek 指定的模型名称
    base_url="https://api.deepseek.com",  # DeepSeek 的 API 端点
    api_key="sk-3e5ff44e82b745a7ab7a748b806951c2",  # 替换为你的 DeepSeek API Key
    temperature=0.5,
    max_tokens=4000
)
# 创建LangChain工具
bocha_tool = Tool(
    name="BochaWebSearch",
    func=bocha_websearch_tool,
    description="使用Bocha Web Search API 进行搜索互联网网页，输入应为搜索查询字符串，输出将返回搜索结果的详细信息，包括网页标题、网页URL、网页摘要、网站名称、网站Icon、网页发布时间等。"
)
tools = [bocha_tool]
llm_with_tools = chat_model.bind_tools(tools)

# 更新聊天机器人节点函数，支持工具调用
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 将更新后的节点添加到状态图中
graph_builder.add_node("chatbot", chatbot)


### 4. 处理工具调用

我们需要创建一个函数来运行工具。我们通过向图中添加一个新节点来实现这一点。



In [6]:
import json
from langchain_core.messages import ToolMessage

# 定义 BasicToolNode，用于执行工具请求
class BasicToolNode:
    """一个在最后一条 AIMessage 中执行工具请求的节点。
    
    该节点会检查最后一条 AI 消息中的工具调用请求，并依次执行这些工具调用。
    """

    def __init__(self, tools: list) -> None:
        # tools 是一个包含所有可用工具的列表，我们将其转化为字典，
        # 通过工具名称（tool.name）来访问具体的工具
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        """执行工具调用
        
        参数:
        inputs: 包含 "messages" 键的字典，"messages" 是对话消息的列表，
                其中最后一条消息可能包含工具调用的请求。
        
        返回:
        包含工具调用结果的消息列表
        """
        # 获取消息列表中的最后一条消息，判断是否包含工具调用请求
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("输入中未找到消息")

        # 用于保存工具调用的结果
        outputs = []

        # 遍历工具调用请求，执行工具并将结果返回
        for tool_call in message.tool_calls:
            # 根据工具名称找到相应的工具，并调用工具的 invoke 方法执行工具
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            # 将工具调用结果作为 ToolMessage 保存下来
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),  # 工具调用的结果以 JSON 格式保存
                    name=tool_call["name"],  # 工具的名称
                    tool_call_id=tool_call["id"],  # 工具调用的唯一标识符
                )
            )
        # 返回包含工具调用结果的消息
        return {"messages": outputs}

In [9]:
# 将 BasicToolNode 添加到状态图中
tool_node = BasicToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

In [10]:
from typing import Literal

# 定义路由函数，检查工具调用
def route_tools(
    state: State,
) -> Literal["tools", "__end__"]:
    """
    使用条件边来检查最后一条消息中是否有工具调用。
    
    参数:
    state: 状态字典或消息列表，用于存储当前对话的状态和消息。
    
    返回:
    如果最后一条消息包含工具调用，返回 "tools" 节点，表示需要执行工具调用；
    否则返回 "__end__"，表示直接结束流程。
    """
    # 检查状态是否是列表类型（即消息列表），取最后一条 AI 消息
    if isinstance(state, list):
        ai_message = state[-1]
    # 否则从状态字典中获取 "messages" 键，取最后一条消息
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    # 如果没有找到消息，则抛出异常
    else:
        raise ValueError(f"输入状态中未找到消息: {state}")

    # 检查最后一条消息是否有工具调用请求
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"  # 如果有工具调用请求，返回 "tools" 节点
    return "__end__"  # 否则返回 "__end__"，流程结束

# 添加条件边，判断是否需要调用工具
graph_builder.add_conditional_edges(
    "chatbot",  # 从聊天机器人节点开始
    route_tools,  # 路由函数，决定下一个节点
    {
        "tools": "tools", 
        "__end__": "__end__"
    },  # 定义条件的输出，工具调用走 "tools"，否则走 "__end__"
)

# 当工具调用完成后，返回到聊天机器人节点以继续对话
graph_builder.add_edge("tools", "chatbot")

# 指定从 START 节点开始，进入聊天机器人节点
graph_builder.add_edge(START, "chatbot")


### 6. 编译图并可视化

使用以下代码可视化构建的状态图：



In [11]:
# 编译状态图，生成可执行的流程图
graph = graph_builder.compile()

In [12]:
from langchain_core.messages import BaseMessage

# 进入一个无限循环，用于模拟持续的对话
while True:
    # 获取用户输入
    user_input = input("User: ")
    
    # 如果用户输入 "quit"、"exit" 或 "q"，则退出循环，结束对话
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")  # 打印告别语
        break  # 退出循环

    # 使用 graph.stream 处理用户输入，并生成机器人的回复
    # "messages" 列表中包含用户的输入，传递给对话系统
    for event in graph.stream({"messages": [("user", user_input)]}):
        
        # 遍历 event 的所有值，检查是否是 BaseMessage 类型的消息
        for value in event.values():
            if isinstance(value["messages"][-1], BaseMessage):
                # 如果消息是 BaseMessage 类型，则打印机器人的回复
                print("Assistant:", value["messages"][-1].content)


User:  火箭队今天赢球了吗


Assistant: 
Assistant: "\u5f15\u7528: 1\n\u6807\u9898: \u706b\u7bad\u8d1f\u6398\u91d1!15\u4eba\u9f50\u767b\u573a,28\u5c81\u5c04\u624b\u906d\u96ea\u85cf,\u4eca\u590f\u5408\u540c\u5230\u671f\u7eed\u7ea6\u592a\u96be\nURL: https://new.qq.com/rain/a/20250414A04LRY00\n\u6458\u8981: \u5317\u4eac\u65f6\u95f44\u670814\u65e5,\u4eca\u5929\u706b\u7bad\u961f\u7ed3\u675f\u4e86\u672c\u8d5b\u5b63\u6700\u540e\u4e00\u573a\u5e38\u89c4\u8d5b\u7684\u8f83\u91cf,\u6700\u7ec8\u706b\u7bad\u961f\u5927\u6bd4\u5206\u8f93\u7ed9\u4e86\u6398\u91d1\u961f,\u8fce\u6765\u4e86\u4e09\u8fde\u8d25\uff61\u5b9e\u9645\u4e0a,\u672c\u573a\u6bd4\u8d5b\u867d\u7136\u706b\u7bad\u961f\u8868\u9762\u4e0a\u5168\u90e8\u4e3b\u529b\u518d\u6b21\u9996\u53d1,\u4f46\u548c\u4e4b\u524d\u4e24\u573a\u6bd4\u8d5b\u7c7b\u4f3c,\u706b\u7bad\u961f\u5e76\u6ca1\u6709\u6253\u7b97\u8d62\u4e0b\u8fd9\u573a\u6bd4\u8d5b\uff61      \u5c3d\u7ba1\u706b\u7bad\u961f\u7684\u4f20\u7edf\u9996\u53d1\u4e94\u864e\u518d\u6b21\u767b\u573a,\u7136\u800c\u4ed6\u4eec\u7684\u51f

User:  q


Goodbye!


------------